In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import VARProcess
import statsmodels.api as sm
import numpy as np
from preproccessing import loadStooqData, loadYahooData



In [2]:
# long rate, short rate, cpi m, real gdp, stock index (SPASTT01USM661N), usdpln, investment growth, consumption growth, unemployment, working population growth,  tradevalue
tickers = ['IRLTLT01USQ156N','IR3TIB01USQ156N','CPALTT01USQ657N','USAGDPRQPSMEI', 'LRUN64TTUSQ156S']

In [3]:
start = datetime.datetime(1992, 10, 1)

end = datetime.datetime(2023, 7, 31)

source_data = web.DataReader(tickers, 'fred', start,end)


In [4]:
data = pd.DataFrame({})
data = source_data.diff().dropna()

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 122 entries, 1993-01-01 to 2023-04-01
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IRLTLT01USQ156N  122 non-null    float64
 1   IR3TIB01USQ156N  122 non-null    float64
 2   CPALTT01USQ657N  122 non-null    float64
 3   USAGDPRQPSMEI    122 non-null    float64
 4   LRUN64TTUSQ156S  122 non-null    float64
dtypes: float64(5)
memory usage: 5.7 KB


In [6]:
for ticker in tickers:
    print(ticker+ ': ', np.round(sm.tsa.stattools.adfuller(data[ticker])[1],4))

IRLTLT01USQ156N:  0.0
IR3TIB01USQ156N:  0.0
CPALTT01USQ657N:  0.0
USAGDPRQPSMEI:  0.0
LRUN64TTUSQ156S:  0.0


In [7]:
model = VAR(data)

c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [8]:
result = model.fit(maxlags=1)
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 17, Aug, 2023
Time:                     22:21:13
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -3.52775
Nobs:                     121.000    HQIC:                  -3.93940
Log likelihood:          -573.092    FPE:                  0.0146911
AIC:                     -4.22092    Det(Omega_mle):       0.0115335
--------------------------------------------------------------------
Results for equation IRLTLT01USQ156N
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                     -0.013268         0.033415           -0.397           0.691
L1.IRLTLT01USQ156N         0.328011         0.105031            3.123           0.002
L1.IR3TIB01USQ156N        -0.1

In [9]:
result.is_stable()

True

In [10]:
for i in range(5):
    print(result.test_causality(caused=i, causing=[0,1,2,3,4]))

<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: ['IRLTLT01USQ156N', 'IR3TIB01USQ156N', 'CPALTT01USQ657N', 'USAGDPRQPSMEI', 'LRUN64TTUSQ156S'] do not Granger-cause IRLTLT01USQ156N: reject at 5% significance level. Test statistic: 2.321, critical value: 2.230>, p-value: 0.042>
<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: ['IRLTLT01USQ156N', 'IR3TIB01USQ156N', 'CPALTT01USQ657N', 'USAGDPRQPSMEI', 'LRUN64TTUSQ156S'] do not Granger-cause IR3TIB01USQ156N: reject at 5% significance level. Test statistic: 19.821, critical value: 2.230>, p-value: 0.000>
<statsmodels.tsa.vector_ar.hypothesis_test_results.CausalityTestResults object. H_0: ['IRLTLT01USQ156N', 'IR3TIB01USQ156N', 'CPALTT01USQ657N', 'USAGDPRQPSMEI', 'LRUN64TTUSQ156S'] do not Granger-cause CPALTT01USQ657N: fail to reject at 5% significance level. Test statistic: 2.027, critical value: 2.230>, p-value: 0.073>
<statsmodels.tsa.vector_ar.hypothesis_test_results.C

In [11]:
result.fevd().summary()

FEVD for IRLTLT01USQ156N
     IRLTLT01USQ156N  IR3TIB01USQ156N  CPALTT01USQ657N  USAGDPRQPSMEI  LRUN64TTUSQ156S
0           1.000000         0.000000         0.000000       0.000000         0.000000
1           0.990037         0.007858         0.000074       0.000001         0.002030
2           0.983620         0.013152         0.000475       0.000269         0.002483
3           0.981328         0.014915         0.000727       0.000374         0.002656
4           0.980768         0.015323         0.000810       0.000412         0.002688
5           0.980665         0.015393         0.000830       0.000419         0.002692
6           0.980651         0.015402         0.000834       0.000421         0.002693
7           0.980649         0.015402         0.000835       0.000421         0.002693
8           0.980649         0.015402         0.000835       0.000421         0.002693
9           0.980649         0.015402         0.000835       0.000421         0.002693

FEVD for IR3TIB01

In [90]:
spy_source = loadStooqData('^SPX','1992-07-1')

In [94]:
spy = spy_source
spy.index = pd.to_datetime(spy.index)
spy = spy.resample('QS').last()
spy = spy.loc['1992-10-01':'2023-04-02']
spy = np.log(spy/spy.shift()).dropna()
#spy = sm.add_constant(spy, prepend=False)
#exog = sm.add_constant(source_data, prepend=False)

In [72]:
bond_source = pd.read_csv('VBMFX.csv')

In [95]:
bond = bond_source
bond = bond.set_index(bond['Date'])
bond = bond['Adj Close']
bond.index = pd.to_datetime(bond.index)
bond = bond.resample('QS').last()
bond = bond.loc['1992-10-01':'2023-04-02']
bond = np.log(bond/bond.shift(1)).dropna()


In [97]:
assets = pd.concat([spy,bond],axis=1)
assets.columns = ['SPY', 'BND']
#assets = assets.diff().dropna()

In [98]:
#np.round(sm.tsa.stattools.adfuller(assets['BND'])[1],4)
sm.tsa.stattools.adfuller(assets['SPY'])

(-10.562787407711081,
 7.65154907320737e-19,
 0,
 121,
 {'1%': -3.485585145896754,
  '5%': -2.885738566292665,
  '10%': -2.5796759080663887},
 -220.56991768257512)

In [99]:
model_a = VAR(assets,exog=data)

c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [100]:
result = model_a.fit(2)

In [101]:
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 17, Aug, 2023
Time:                     23:53:23
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -12.8087
Nobs:                     120.000    HQIC:                  -13.0846
Log likelihood:           475.853    FPE:                1.72112e-06
AIC:                     -13.2733    Det(Omega_mle):     1.46651e-06
--------------------------------------------------------------------
Results for equation SPY
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                  -0.000419         0.009632           -0.043           0.965
IRLTLT01USQ156N         0.066201         0.026863            2.464           0.014
IR3TIB01USQ156N         0.035644         0.021916     

In [103]:
result.is_stable()

True